# Example: Apple 2020

In [1]:
import logging
from openesef.util.util_mylogger import setup_logger 
logger = setup_logger("main", logging.INFO, log_dir="/tmp/log/")

In [2]:
from openesef.edgar.loader import load_xbrl_filing
xid, tax = load_xbrl_filing(ticker="AAPL", year=2020)

2025-03-02 23:55:46,676 - main.openesf.base.pool - PID:59253 - INFO - 

Initializing Pool with cache_folder=None, output_folder=None
2025-03-02 23:55:46,677 - main.openesf.base.pool - PID:59253 - INFO - Using repository cache folder: /Users/mbp16/Dropbox/sciebo/WebScraping+ESEF_Paper/Research/code_fse/openesef_repo/openesef/xbrl_schema
2025-03-02 23:55:46,923 - main.openesf.taxonomy.linkbase - PID:59253 - ERROR - Failed to load linkbase: location=/Users/mbp16/Dropbox/sciebo/WebScraping%2BESEF_Paper/Research/code_fse/openesef_repo/examples/mem%3A/aapl-20200926_cal.xml, esef_filing_root=mem:// 
Error reading file '/Users/mbp16/Dropbox/sciebo/WebScraping%2BESEF_Paper/Research/code_fse/openesef_repo/examples/mem%3A/aapl-20200926_cal.xml': failed to load external entity "/Users/mbp16/Dropbox/sciebo/WebScraping%2BESEF_Paper/Research/code_fse/openesef_repo/examples/mem%3A/aapl-20200926_cal.xml"
2025-03-02 23:55:46,923 - main.openesf.taxonomy.linkbase - PID:59253 - ERROR - Failed to load linkb

In [3]:
print(xid)


Namespaces: 10
Schema references: 1
Linkbase references: 0
Contexts: 318
Units: 9
Facts: 1388
Footnotes: 0
Filing Indicators: 0


In [4]:
print(tax)

Schemas: 15
Linkbases: 15
Role Types: 775
Arcrole Types: 0
Concepts: 18391
Item Types: 52
Tuple Types: 0
Simple Types: 0
Labels: 1527
References: 0
Hierarchies: 239
Dimensional Relationship Sets: 239
Dimensions: 295
Hypercubes: 366
Enumerations: 0
Enumerations Sets: 0
Table Groups: 0
Tables: 0
Parameters: 0
Assertion Sets: 0
Value Assertions: 0
Existence Assertions: 0
Consistency Assertions: 0


In [5]:
for i, (key, value) in enumerate(xid.dei.items()):
    print(f"{i}: {key}: {value}")


0: AmendmentFlag: false
1: DocumentFiscalYearFocus: 2020
2: DocumentFiscalPeriodFocus: FY
3: EntityCentralIndexKey: 0000320193
4: CurrentFiscalYearEndDate: --09-26
5: DocumentType: 10-K
6: DocumentAnnualReport: true
7: DocumentPeriodEndDate: 2020-09-26
8: DocumentTransitionReport: false
9: EntityFileNumber: 001-36743
10: EntityRegistrantName: Apple Inc.
11: EntityIncorporationStateCountryCode: CA
12: EntityTaxIdentificationNumber: 94-2404110
13: EntityAddressAddressLine1: One Apple Park Way
14: EntityAddressCityOrTown: Cupertino
15: EntityAddressStateOrProvince: CA
16: EntityAddressPostalZipCode: 95014
17: CityAreaCode: 408
18: LocalPhoneNumber: 996-1010
19: TradingSymbol: AAPL
20: EntityWellKnownSeasonedIssuer: Yes
21: EntityVoluntaryFilers: No
22: EntityCurrentReportingStatus: Yes
23: EntityInteractiveDataCurrent: Yes
24: EntityFilerCategory: Large Accelerated Filer
25: EntitySmallBusiness: false
26: EntityEmergingGrowthCompany: false
27: IcfrAuditorAttestationFlag: true
28: EntitySh